In [1]:
# Import necessary libraries

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

# Check TensorFlow version
print(tf.__version__)

2.14.0


In [4]:
# Set datasets

train_data = 'dataset/Training'
test_data = 'dataset/Testing'

### Define image data generators

**rescale=1./255:**

- **Purpose**: This rescales the pixel values of the images from a range of [0, 255] to [0, 1].
- **Reason**: Neural networks perform better when input data is normalized. Scaling the pixel values to this range helps in faster convergence during training.

**rotation_range=40:**

- **Purpose**: Randomly rotates images within a range of 0 to 40 degrees.
- **Reason**: Rotation helps the model become invariant to the orientation of the images, making it more robust to variations in the dataset.

**width_shift_range=0.2:**

- **Purpose**: Shifts the image horizontally by up to 20% of the width.
- **Reason**: Horizontal shifts help the model to learn that objects can appear in different horizontal positions, enhancing its generalization capability.

**height_shift_range=0.2:**

- **Purpose**: Shifts the image vertically by up to 20% of the height.
- **Reason**: Vertical shifts serve the same purpose as horizontal shifts but in the vertical direction, improving the model's robustness to vertical translations.

**shear_range=0.2:**

- **Purpose**: Applies a shear transformation to the images, up to 20%.
- **Reason**: Shearing distorts the image along an axis, helping the model handle slight distortions or slanting of the objects within the images.

**zoom_range=0.2:**

- **Purpose**: Randomly zooms into the images by up to 20%.
- **Reason**: Zooming helps the model become invariant to the scale of the objects in the images, allowing it to detect objects at different zoom levels.

**horizontal_flip=True:**

- **Purpose**: Randomly flips the images horizontally.
- **Reason**: Horizontal flipping augments the dataset by generating mirrored versions of the images, which helps the model learn that objects can appear mirrored.

**fill_mode='nearest':**

- **Purpose**: Specifies the strategy for filling in new pixels that may be created during transformations (e.g., after rotation or shifts).
- **Reason**: 'nearest' mode fills in the new pixels with the nearest pixel value, which helps maintain the integrity of the image after augmentation.


In [21]:
# Define image data generators

train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40, 
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

In [22]:
test_datagen = ImageDataGenerator(rescale=1./255)

### Create generators


**train_generator = train_datagen.flow_from_directory(...):**

**train_dir:**

- **Purpose**: Specifies the path to the directory containing the training images organized in subdirectories by class.

**target_size=(150,150):**

- **Purpose**: Resizes all images to the specified dimensions (150 pixels by 150 pixels).

**batch_size=32:**

- **Purpose**: Specifies the number of images to be yielded from the generator per batch

**class_mode='categorical':**

- **Purpose**: Specifies the type of label arrays to be returned. 'categorical' indicates that the labels are one-hot encoded (e.g., [1, 0, 0, 0] for class 0 in a 4-class problem).

This is also same as the `test_generator`

In [23]:
train_generator = train_datagen.flow_from_directory(
    train_data,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical')

Found 2870 images belonging to 4 classes.


In [24]:
test_generator = test_datagen.flow_from_directory(
    test_data,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical')

Found 394 images belonging to 4 classes.


### Defining the CNN model

In [25]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(4, activation='softmax')  # Change here for 4 classes
])

model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d_8 (MaxPoolin  (None, 74, 74, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_9 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_9 (MaxPoolin  (None, 36, 36, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_10 (Conv2D)          (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_10 (MaxPooli  (None, 17, 17, 128)      

### Training the model

In [26]:
# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=30,
    validation_data=test_generator,
    validation_steps=test_generator.samples // test_generator.batch_size,
    callbacks=[early_stopping]
)


Epoch 1/30
89/89 [==============================] - 53s 584ms/step - loss: 1.3028 - accuracy: 0.3643 - val_loss: 1.4568 - val_accuracy: 0.1458
Epoch 2/30
89/89 [==============================] - 49s 549ms/step - loss: 1.2033 - accuracy: 0.4454 - val_loss: 1.6517 - val_accuracy: 0.2734
Epoch 3/30
89/89 [==============================] - 49s 553ms/step - loss: 1.0452 - accuracy: 0.5444 - val_loss: 2.0963 - val_accuracy: 0.2604
Epoch 4/30
89/89 [==============================] - 49s 550ms/step - loss: 0.9897 - accuracy: 0.5638 - val_loss: 2.5457 - val_accuracy: 0.3203
Epoch 5/30
89/89 [==============================] - 49s 547ms/step - loss: 0.9253 - accuracy: 0.5980 - val_loss: 2.6036 - val_accuracy: 0.2526
Epoch 6/30
89/89 [==============================] - 51s 574ms/step - loss: 0.8841 - accuracy: 0.6209 - val_loss: 2.2432 - val_accuracy: 0.3776


### Evaluate the model

In [27]:
loss, accuracy = model.evaluate(test_generator)
print(f'Test accuracy: {accuracy * 100:.2f}%')

13/13 [==============================] - 1s 111ms/step - loss: 1.4593 - accuracy: 0.1472
Test accuracy: 14.72%


In [13]:
# Save the model
model.save('tumor_detection_model.h5')

C:\Users\Angel\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
